# Basic Settings

In [0]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import pandas as pd
import numpy as np

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
path = 'drive/My Drive/bigcontest2019/data/'
print(os.listdir(path + 'train'),'\n', 
      os.listdir(path + 'test'),'\n', 
    #   os.listdir(path + '../metrics'),'\n', 
      os.listdir(path + '../scripts'))

['train_payment.csv', 'train_trade.csv', 'train_pledge.csv', 'train_combat.csv', 'train_label.csv', 'train_activity.csv', 'train_merge_bjw.csv'] 
 ['test1_activity.csv', 'test1_combat.csv', 'test2_activity.csv', 'test1_pledge.csv', 'test2_combat.csv', 'test2_trade.csv', 'test1_trade.csv', 'test1_payment.csv', 'test2_payment.csv', 'test2_pledge.csv'] 
 ['explortary data', 'model', 'data merge']


# train

In [0]:
# train = pd.read_csv(path+'train/train_merge_bjw.csv').iloc[:,1:].reset_index(drop=True)

In [0]:
import pandas as pd
import numpy as np
import os
import sys

# data load
path = 'drive/My Drive/bigcontest2019/data/'


train_label = pd.read_csv(path + 'train/train_label.csv')
train_activity = pd.read_csv(path + 'train/train_activity.csv')
train_payment = pd.read_csv(path + 'train/train_payment.csv')
train_trade = pd.read_csv(path + 'train/train_trade.csv')
train_pledge = pd.read_csv(path + 'train/train_pledge.csv')
train_combat = pd.read_csv(path + 'train/train_combat.csv')

# activity
train_activity['game_money_change'] = np.abs(train_activity['game_money_change'])
train_activity_group = train_activity.groupby('acc_id').agg({'day':'nunique', 
                                                             'char_id':'nunique', 
                                                             'server':'nunique', 
                                                             'playtime':['sum','mean'], 
                                                             'npc_kill':['sum','mean'], 
                                                             'solo_exp':'sum',
                                                             'party_exp':'sum', 
                                                             'quest_exp':'sum',
                                                             'rich_monster':'sum', 
                                                             'death':'sum', 
                                                             'revive':'sum',
                                                             'exp_recovery':'sum',
                                                             'fishing':'sum',
                                                             'private_shop':'sum',
                                                             'game_money_change':'sum',
                                                             'enchant_count':'sum'})
train_activity_group.columns = ['activity_'+'_'.join(x) for x in train_activity_group.columns.ravel()]


# payment
train_payment_group = train_payment.groupby('acc_id').agg({'day':'nunique',
                                                           'amount_spent':'sum'})
train_payment_group.columns = ['payment_'+i for i in train_payment_group.columns]


# trade
## 판매자 테이블 정의
grouped_trade_seller = train_trade.groupby('source_acc_id').agg({'day':'count', 
                                         'type':['nunique','count','sum'], # 거래의 종류들을 파악하기 위해서 -> nuique = 2이면 두 종류의 거래 모두 진행 / count 
                                         'server':'nunique', 
                                         'source_char_id':'nunique', # 몇개의 캐릭터 운용하는지
                                         'target_acc_id':'nunique', 
                                         'target_char_id':'nunique',
                                         'item_type':'nunique', 
                                         'item_amount':'sum',
                                         'item_price':'sum'})

grouped_trade_seller.columns = ['trade_seller_'+'_'.join(x) for x in grouped_trade_seller.columns.ravel()]
grouped_trade_seller['trade_seller_type_count'] -= grouped_trade_seller['trade_seller_type_sum']
grouped_trade_seller = grouped_trade_seller.rename(columns = {'trade_seller_type_count':'trade_seller_type_personal','trade_seller_type_sum':'trade_seller_type_exchange'})


## 구매자 테이블 정의
grouped_trade_buyer = train_trade.groupby('target_acc_id').agg({'day':'count', 
                                         'type':['nunique','count','sum'], # 거래의 종류들을 파악하기 위해서 -> nuique = 2이면 두 종류의 거래 모두 진행 / count 
                                         'server':'nunique', 
                                        'target_char_id':'nunique',
                                         'source_acc_id':'nunique', 
                                         'source_char_id':'nunique', # 몇개의 캐릭터 운용하는지
                                         'item_type':'nunique', 
                                         'item_amount':'sum',
                                          'item_price':'sum'})
grouped_trade_buyer.columns = ['trade_buyer_'+'_'.join(x) for x in grouped_trade_buyer.columns.ravel()]
grouped_trade_buyer['trade_buyer_type_count'] -= grouped_trade_buyer['trade_buyer_type_sum']
grouped_trade_buyer = grouped_trade_buyer.rename(columns = {'trade_buyer_type_count':'trade_buyer_type_personal','trade_buyer_type_sum':'trade_buyer_type_exchange'})

## trade merge
train_trade_group = pd.concat([grouped_trade_seller, grouped_trade_buyer], axis=1)


# pledge
train_pledge_group = train_pledge.groupby('acc_id').agg(
    {
        'day': 'nunique',                   # 날짜
        'char_id': 'nunique',               # 캐릭터 아이디
        'server' : 'nunique',               # 캐릭터 서버
        'pledge_id' : 'nunique',            # 혈맹 아이디
        'play_char_cnt' : 'sum',        # 게임에 접속한 혈맹원 수
        'combat_char_cnt' : 'sum',      # 전투에 참여한 혈맹원 수
        'pledge_combat_cnt': 'sum',     # 혈맹 간 전투 횟수의 합
        'random_attacker_cnt' : 'sum',  # 혈맹원 중 막피 전투를 행한 횟수의 합
        'random_defender_cnt': 'sum',   # 혈맹원 중 막피로부터 피해를 받은 횟수의 합
        'same_pledge_cnt': 'sum',       # 동일 혈맹원 간 전투 횟수의 합
        'temp_cnt' : 'sum',             # 혈맹원들의 단발성 전투 횟수의 합
        'etc_cnt' : 'sum',              # 혈맹원들의 기타 전투 횟수의 합
        'combat_play_time': 'sum',      # 혈맹의 전투 캐릭터들의 플레이 시간의 합
        'non_combat_play_time' : 'sum' # 혈맹의 非전투 캐릭터 플레이 시간의 합
    })

train_pledge_group.columns = ['pledge_'+i for i in train_pledge_group.columns]


# combat
train_combat_group = train_combat.groupby('acc_id').agg(
    {
        'day': 'nunique',              # 날짜
        'char_id': 'nunique',          # 캐릭터 아이디
        'server' : 'nunique',          # 캐릭터 서버
        'class' : 'nunique',           # 직업
        'pledge_cnt' : 'sum',          # 혈맹간 전투에 참여한 횟수
        'random_attacker_cnt' : 'sum', # 본인이 막피 공격을 행한 횟수
        'random_defender_cnt' : 'sum', # 막피 공격자로부터 공격을 받은 횟수
        'temp_cnt' : 'sum',            # 단발성 전투 횟수
        'same_pledge_cnt' : 'sum',     # 동일 혈맹원 간의 전투 횟수
        'etc_cnt' : 'sum',             # 기타 전투 횟수
        'num_opponent' : 'sum'         # 전투 상대 캐릭터수
    })
train_combat_group.columns = ['combat_'+i for i in train_combat_group.columns]

def merge_all_df(df, by ,*args):
    '''
    df : 기준이 되는 데이터 프레임(left join)
    by : 기준이 되는 열 (df 외 타 args(df) 에도 해당 열 존재하게 생성할 것)
    *args : 가변인자 (데이터 프레임을 원하는 만큼 집어넣으면 된다.)
    '''    
    for arg in args:
        if by not in arg.columns:
            arg[by] = arg.index
            del arg.index.name
        df = pd.merge(df, arg, how='left',on= by)
    return df



In [0]:
train = merge_all_df(train_label,'acc_id',train_trade_group,
                     train_payment_group,train_activity_group,
                     train_combat_group,train_pledge_group).fillna(0).reset_index(drop=True)


In [0]:
train.shape

(40000, 70)

# test1

In [0]:
import pandas as pd
import numpy as np
import os
import sys

# data load

test1_activity = pd.read_csv(path + 'test/test1_activity.csv')
test1_payment = pd.read_csv(path + 'test/test1_payment.csv')
test1_trade = pd.read_csv(path + 'test/test1_trade.csv')
test1_pledge = pd.read_csv(path + 'test/test1_pledge.csv')
test1_combat = pd.read_csv(path + 'test/test1_combat.csv')


# activity
test1_activity['game_money_change'] = np.abs(test1_activity['game_money_change'])
test1_activity_group = test1_activity.groupby('acc_id').agg({'day':'nunique', 
                                                             'char_id':'nunique', 
                                                             'server':'nunique', 
                                                             'playtime':['sum','mean'], 
                                                             'npc_kill':['sum','mean'], 
                                                             'solo_exp':'sum',
                                                             'party_exp':'sum', 
                                                             'quest_exp':'sum',
                                                             'rich_monster':'sum', 
                                                             'death':'sum', 
                                                             'revive':'sum',
                                                             'exp_recovery':'sum',
                                                             'fishing':'sum',
                                                             'private_shop':'sum',
                                                             'game_money_change':'sum',
                                                             'enchant_count':'sum'})
test1_activity_group.columns = ['activity_'+'_'.join(x) for x in test1_activity_group.columns.ravel()]


# payment
test1_payment_group = test1_payment.groupby('acc_id').agg({'day':'nunique',
                                                           'amount_spent':'sum'})
test1_payment_group.columns = ['payment_'+i for i in test1_payment_group.columns]


# trade
## 판매자 테이블 정의
grouped_trade_seller = test1_trade.groupby('source_acc_id').agg({'day':'count', 
                                         'type':['nunique','count','sum'], # 거래의 종류들을 파악하기 위해서 -> nuique = 2이면 두 종류의 거래 모두 진행 / count 
                                         'server':'nunique', 
                                         'source_char_id':'nunique', # 몇개의 캐릭터 운용하는지
                                         'target_acc_id':'nunique', 
                                         'target_char_id':'nunique',
                                         'item_type':'nunique', 
                                         'item_amount':'sum',
                                         'item_price':'sum'})

grouped_trade_seller.columns = ['trade_seller_'+'_'.join(x) for x in grouped_trade_seller.columns.ravel()]
grouped_trade_seller['trade_seller_type_count'] -= grouped_trade_seller['trade_seller_type_sum']
grouped_trade_seller = grouped_trade_seller.rename(columns = {'trade_seller_type_count':'trade_seller_type_personal','trade_seller_type_sum':'trade_seller_type_exchange'})


## 구매자 테이블 정의
grouped_trade_buyer = test1_trade.groupby('target_acc_id').agg({'day':'count', 
                                         'type':['nunique','count','sum'], # 거래의 종류들을 파악하기 위해서 -> nuique = 2이면 두 종류의 거래 모두 진행 / count 
                                         'server':'nunique', 
                                        'target_char_id':'nunique',
                                         'source_acc_id':'nunique', 
                                         'source_char_id':'nunique', # 몇개의 캐릭터 운용하는지
                                         'item_type':'nunique', 
                                         'item_amount':'sum',
                                          'item_price':'sum'})

grouped_trade_buyer.columns = ['trade_buyer_'+'_'.join(x) for x in grouped_trade_buyer.columns.ravel()]
grouped_trade_buyer['trade_buyer_type_count'] -= grouped_trade_buyer['trade_buyer_type_sum']
grouped_trade_buyer = grouped_trade_buyer.rename(columns = {'trade_buyer_type_count':'trade_buyer_type_personal','trade_buyer_type_sum':'trade_buyer_type_exchange'})

## trade merge
test1_trade_group = pd.concat([grouped_trade_seller, grouped_trade_buyer], axis=1)


# pledge
test1_pledge_group = test1_pledge.groupby('acc_id').agg(
    {
        'day': 'nunique',                   # 날짜
        'char_id': 'nunique',               # 캐릭터 아이디
        'server' : 'nunique',               # 캐릭터 서버
        'pledge_id' : 'nunique',            # 혈맹 아이디
        'play_char_cnt' : 'sum',        # 게임에 접속한 혈맹원 수
        'combat_char_cnt' : 'sum',      # 전투에 참여한 혈맹원 수
        'pledge_combat_cnt': 'sum',     # 혈맹 간 전투 횟수의 합
        'random_attacker_cnt' : 'sum',  # 혈맹원 중 막피 전투를 행한 횟수의 합
        'random_defender_cnt': 'sum',   # 혈맹원 중 막피로부터 피해를 받은 횟수의 합
        'same_pledge_cnt': 'sum',       # 동일 혈맹원 간 전투 횟수의 합
        'temp_cnt' : 'sum',             # 혈맹원들의 단발성 전투 횟수의 합
        'etc_cnt' : 'sum',              # 혈맹원들의 기타 전투 횟수의 합
        'combat_play_time': 'sum',      # 혈맹의 전투 캐릭터들의 플레이 시간의 합
        'non_combat_play_time' : 'sum' # 혈맹의 非전투 캐릭터 플레이 시간의 합
    })

test1_pledge_group.columns = ['pledge_'+i for i in test1_pledge_group.columns]


# combat
test1_combat_group = test1_combat.groupby('acc_id').agg(
    {
        'day': 'nunique',              # 날짜
        'char_id': 'nunique',          # 캐릭터 아이디
        'server' : 'nunique',          # 캐릭터 서버
        'class' : 'nunique',           # 직업
     
        'pledge_cnt' : 'sum',          # 혈맹간 전투에 참여한 횟수
        'random_attacker_cnt' : 'sum', # 본인이 막피 공격을 행한 횟수
        'random_defender_cnt' : 'sum', # 막피 공격자로부터 공격을 받은 횟수
        'temp_cnt' : 'sum',            # 단발성 전투 횟수
        'same_pledge_cnt' : 'sum',     # 동일 혈맹원 간의 전투 횟수
        'etc_cnt' : 'sum',             # 기타 전투 횟수
        'num_opponent' : 'sum'         # 전투 상대 캐릭터수
    })
test1_combat_group.columns = ['combat_'+i for i in test1_combat_group.columns]
                    

In [0]:

def merge_test_df(df,*args):
    '''
    by : 기준이 되는 열 (df 외 타 args(df) 에도 해당 열 존재하게 생성할 것)
    *args : 가변인자 (데이터 프레임을 원하는 만큼 집어넣으면 된다.)
    '''    
    df = df.reset_index(drop=False)
    df = df.rename(columns = {'index': 'acc_id'})
    
    for arg in args:
        arg = arg.reset_index(drop=False)
        arg = arg.rename(columns = {'index': 'acc_id'})
        df = pd.merge(df, arg, how='left',on= 'acc_id')
    
    return df

In [0]:
test1 = merge_test_df(test1_activity_group,
                      test1_payment_group,
                     test1_combat_group,
                      test1_pledge_group,
                      test1_trade_group).fillna(0)

In [0]:
train.shape

(40000, 70)

In [0]:
test1.shape

(20000, 68)

In [0]:
# [ i for i in test1.columns if i not in train.columns]

# test 2

In [0]:
import pandas as pd
import numpy as np
import os
import sys

# data load
test2_activity = pd.read_csv(path + 'test/test2_activity.csv')
test2_payment = pd.read_csv(path + 'test/test2_payment.csv')
test2_trade = pd.read_csv(path + 'test/test2_trade.csv')
test2_pledge = pd.read_csv(path + 'test/test2_pledge.csv')
test2_combat = pd.read_csv(path + 'test/test2_combat.csv')


# activity
test2_activity['game_money_change'] = np.abs(test2_activity['game_money_change'])
test2_activity_group = test2_activity.groupby('acc_id').agg({'day':'nunique', 
                                                             'char_id':'nunique', 
                                                             'server':'nunique', 
                                                             'playtime':['sum','mean'], 
                                                             'npc_kill':['sum','mean'], 
                                                             'solo_exp':'sum',
                                                             'party_exp':'sum', 
                                                             'quest_exp':'sum',
                                                             'rich_monster':'sum', 
                                                             'death':'sum', 
                                                             'revive':'sum',
                                                             'exp_recovery':'sum',
                                                             'fishing':'sum',
                                                             'private_shop':'sum',
                                                             'game_money_change':'sum',
                                                             'enchant_count':'sum'})
test2_activity_group.columns = ['activity_'+'_'.join(x) for x in test2_activity_group.columns.ravel()]


# payment
test2_payment_group = test2_payment.groupby('acc_id').agg({'day':'nunique',
                                                           'amount_spent':'sum'})
test2_payment_group.columns = ['payment_'+i for i in test2_payment_group.columns]


# trade
## 판매자 테이블 정의
grouped_trade_seller = test2_trade.groupby('source_acc_id').agg({'day':'count', 
                                         'type':['nunique','count','sum'], # 거래의 종류들을 파악하기 위해서 -> nuique = 2이면 두 종류의 거래 모두 진행 / count 
                                         'server':'nunique', 
                                         'source_char_id':'nunique', # 몇개의 캐릭터 운용하는지
                                         'target_acc_id':'nunique', 
                                         'target_char_id':'nunique',
                                         'item_type':'nunique', 
                                         'item_amount':'sum',
                                         'item_price':'sum'})

grouped_trade_seller.columns = ['trade_seller_'+'_'.join(x) for x in grouped_trade_seller.columns.ravel()]
grouped_trade_seller['trade_seller_type_count'] -= grouped_trade_seller['trade_seller_type_sum']
grouped_trade_seller = grouped_trade_seller.rename(columns = {'trade_seller_type_count':'trade_seller_type_personal','trade_seller_type_sum':'trade_seller_type_exchange'})


## 구매자 테이블 정의
grouped_trade_buyer = test2_trade.groupby('target_acc_id').agg({'day':'count', 
                                         'type':['nunique','count','sum'], # 거래의 종류들을 파악하기 위해서 -> nuique = 2이면 두 종류의 거래 모두 진행 / count 
                                         'server':'nunique', 
                                        'target_char_id':'nunique',
                                         'source_acc_id':'nunique', 
                                         'source_char_id':'nunique', # 몇개의 캐릭터 운용하는지
                                         'item_type':'nunique', 
                                         'item_amount':'sum',
                                          'item_price':'sum'})
grouped_trade_buyer.columns = ['trade_buyer_'+'_'.join(x) for x in grouped_trade_buyer.columns.ravel()]
grouped_trade_buyer['trade_buyer_type_count'] -= grouped_trade_buyer['trade_buyer_type_sum']
grouped_trade_buyer = grouped_trade_buyer.rename(columns = {'trade_buyer_type_count':'trade_buyer_type_personal','trade_buyer_type_sum':'trade_buyer_type_exchange'})

## trade merge
test2_trade_group = pd.concat([grouped_trade_seller, grouped_trade_buyer], axis=1)


# pledge
test2_pledge_group = test2_pledge.groupby('acc_id').agg(
    {
        'day': 'nunique',                   # 날짜
        'char_id': 'nunique',               # 캐릭터 아이디
        'server' : 'nunique',               # 캐릭터 서버
        'pledge_id' : 'nunique',            # 혈맹 아이디
        'play_char_cnt' : 'sum',        # 게임에 접속한 혈맹원 수
        'combat_char_cnt' : 'sum',      # 전투에 참여한 혈맹원 수
        'pledge_combat_cnt': 'sum',     # 혈맹 간 전투 횟수의 합
        'random_attacker_cnt' : 'sum',  # 혈맹원 중 막피 전투를 행한 횟수의 합
        'random_defender_cnt': 'sum',   # 혈맹원 중 막피로부터 피해를 받은 횟수의 합
        'same_pledge_cnt': 'sum',       # 동일 혈맹원 간 전투 횟수의 합
        'temp_cnt' : 'sum',             # 혈맹원들의 단발성 전투 횟수의 합
        'etc_cnt' : 'sum',              # 혈맹원들의 기타 전투 횟수의 합
        'combat_play_time': 'sum',      # 혈맹의 전투 캐릭터들의 플레이 시간의 합
        'non_combat_play_time' : 'sum' # 혈맹의 非전투 캐릭터 플레이 시간의 합
    })

test2_pledge_group.columns = ['pledge_'+i for i in test2_pledge_group.columns]


# combat
test2_combat_group = test2_combat.groupby('acc_id').agg(
    {
        'day': 'nunique',              # 날짜
        'char_id': 'nunique',          # 캐릭터 아이디
        'server' : 'nunique',          # 캐릭터 서버
        'class' : 'nunique',           # 직업
        'pledge_cnt' : 'sum',          # 혈맹간 전투에 참여한 횟수
        'random_attacker_cnt' : 'sum', # 본인이 막피 공격을 행한 횟수
        'random_defender_cnt' : 'sum', # 막피 공격자로부터 공격을 받은 횟수
        'temp_cnt' : 'sum',            # 단발성 전투 횟수
        'same_pledge_cnt' : 'sum',     # 동일 혈맹원 간의 전투 횟수
        'etc_cnt' : 'sum',             # 기타 전투 횟수
        'num_opponent' : 'sum'         # 전투 상대 캐릭터수
    })
test2_combat_group.columns = ['combat_'+i for i in test2_combat_group.columns]

In [0]:
test2 = merge_test_df(test2_combat_group,test2_trade_group,
                     test2_payment_group,test2_activity_group,
                     test2_pledge_group).fillna(0)

In [0]:
test2.shape

(20000, 68)

In [0]:
test2.head()

,acc_id,combat_day,combat_char_id,combat_server,combat_class,combat_pledge_cnt,combat_random_attacker_cnt,combat_random_defender_cnt,combat_temp_cnt,combat_same_pledge_cnt,combat_etc_cnt,combat_num_opponent,trade_seller_day_count,trade_seller_type_nunique,trade_seller_type_personal,trade_seller_type_exchange,trade_seller_server_nunique,trade_seller_source_char_id_nunique,trade_seller_target_acc_id_nunique,trade_seller_target_char_id_nunique,trade_seller_item_type_nunique,trade_seller_item_amount_sum,trade_seller_item_price_sum,trade_buyer_day_count,trade_buyer_type_nunique,trade_buyer_type_personal,trade_buyer_type_exchange,trade_buyer_server_nunique,trade_buyer_target_char_id_nunique,trade_buyer_source_acc_id_nunique,trade_buyer_source_char_id_nunique,trade_buyer_item_type_nunique,trade_buyer_item_amount_sum,trade_buyer_item_price_sum,payment_day,payment_amount_spent,activity_day_nunique,activity_char_id_nunique,activity_server_nunique,activity_playtime_sum,activity_playtime_mean,activity_npc_kill_sum,activity_npc_kill_mean,activity_solo_exp_sum,activity_party_exp_sum,activity_quest_exp_sum,activity_rich_monster_sum,activity_death_sum,activity_revive_sum,activity_exp_recovery_sum,activity_fishing_sum,activity_private_shop_sum,activity_game_money_change_sum,activity_enchant_count_sum,pledge_day,pledge_char_id,pledge_server,pledge_pledge_id,pledge_play_char_cnt,pledge_combat_char_cnt,pledge_pledge_combat_cnt,pledge_random_attacker_cnt,pledge_random_defender_cnt,pledge_same_pledge_cnt,pledge_temp_cnt,pledge_etc_cnt,pledge_combat_play_time,pledge_non_combat_play_time
0,1,28,2,1,2,0.000000,0.0,80.448385,0.479728,0.000000,37.499638,18.840721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.0,0.348521,28,2,1,88.551668,2.012538,58.285882,1.324679,0.512386,0.000000,0.000000,0,2.704718,0.000000,0.0,0.000000,0.0,0.003433,0.0,28.0,1.0,1.0,1.0,10.646349,9.744116,0.000000,0.000000,43.562585,0.000000,0.889947,13.080634,31.973768,0.000000
1,3,21,9,5,5,0.000000,0.0,1.183064,0.000000,0.000000,0.112612,0.196258,4.0,1.0,0.0,4.0,2.0,3.0,3.0,3.0,3.0,9.588032e-02,0.0,6.0,1.0,6.0,0.0,1.0,2.0,6.0,6.0,1.0,0.000003,14.491608,5.0,5.984698,21,9,5,2.694094,0.081639,0.092596,0.002806,0.010214,0.000000,0.078912,2,0.000000,0.000000,0.0,1.025441,0.0,0.051625,0.0,9.0,2.0,1.0,1.0,0.938322,0.072179,0.000000,0.000000,0.000000,0.000000,0.395532,0.000000,0.012278,0.050734
2,9,4,4,1,3,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,1.0,0.0,3.0,1.0,1.0,1.0,1.0,3.0,3.835174e-07,0.0,10.0,1.0,0.0,10.0,1.0,2.0,2.0,3.0,5.0,0.023971,0.000000,2.0,1.756685,4,4,1,0.081923,0.016385,0.015883,0.003177,0.005888,0.000000,0.066681,0,0.000000,0.000000,0.0,0.000000,0.0,0.028409,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,14,12,8,1,5,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,53.0,1.0,0.0,53.0,1.0,1.0,2.0,3.0,6.0,4.813144e-03,0.0,56.0,2.0,1.0,55.0,1.0,1.0,3.0,4.0,4.0,0.000063,0.033485,2.0,5.860310,12,8,2,6.558516,0.142576,0.215944,0.004694,0.033683,0.000837,0.006073,4,0.983534,0.989348,0.0,0.526569,0.0,0.052707,0.0,12.0,7.0,1.0,5.0,20.787448,5.160773,0.675690,8.424075,0.858376,3.978586,6.921813,6.327969,15.673214,1.173368
4,26,28,6,3,4,66.687428,0.0,3.549193,2.878367,24.796545,3.265734,78.110489,137.0,1.0,0.0,137.0,1.0,2.0,10.0,12.0,7.0,1.021155e+00,0.0,143.0,2.0,24.0,119.0,1.0,1.0,30.0,31.0,7.0,1.793944,2.597867,2.0,1.405817,28,6,4,31.816526,0.435843,3.774791,0.051709,0.428995,0.009611,0.000868,5,29.506016,29.680428,0.0,83.157199,0.0,5.968221,0.0,28.0,4.0,3.0,4.0,79.071699,48.937117,98.543341,44.767020,16.881396,59.280931,50.133705,19.918239,34.446761,0.085020


# Model

In [0]:
# model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge 

import xgboost as xgb
import warnings

class model(object):
    def __init__(self, train, test, label, folds=FOLDS, seed=SEED, log=True):
        self.train = train
        self.test = test #.drop(['amount_spent','survival_time'], axis=1)
        self.label = label
        self.kf = KFold(n_splits=folds, random_state=seed)
        self.log = log
    
    def ridge_model(self):
        oof = np.zeros(len(self.train))
        pred = np.zeros(len(self.test))
        for trn_idx, val_idx in self.kf.split(self.train):
            train_df = self.train.loc[trn_idx]
            valid_df = self.train.loc[val_idx].drop(['amount_spent','survival_time'], axis=1)
            if self.log:
                ridge_model = Ridge().fit(train_df.drop(['amount_spent','survival_time'], axis=1), np.log1p(train_df[self.label]))
            else:
                ridge_model = Ridge().fit(train_df.drop(['amount_spent','survival_time'], axis=1), train_df[self.label])
            oof[val_idx] = ridge_model.predict(valid_df)
            pred += ridge_model.predict(self.test)/self.kf.n_splits
        return oof, pred
    
    def rf_model(self):
        
        oof = np.zeros(len(self.train))
        pred = np.zeros(len(self.test))
        for trn_idx, val_idx in self.kf.split(self.train):
            train_df = self.train.loc[trn_idx]
            valid_df = self.train.loc[val_idx].drop(['amount_spent','survival_time'], axis=1)
            if self.log:
                rf = RandomForestRegressor().fit(X=train_df.drop(['amount_spent','survival_time'], axis=1), y=np.log1p(train_df[self.label]))
            else:
                rf = RandomForestRegressor().fit(X= train_df.drop(['amount_spent','survival_time'], axis=1), y = train_df[self.label])
            oof[val_idx] = rf.predict(valid_df)
            pred += rf.predict(self.test)/self.kf.n_splits
        return oof, pred
        
    def xgb_model(self):
        warnings.filterwarnings(action='ignore')
        
        params = {
            'objective':'survival:cox',
            'n_estimators':10000,
            'max_depth':16,
            'learning_rate':0.001,
            'subsample':0.7,
            'colsample_bytree':0.7,
            'reg_alpha':0.2,
            'tree_method':'gpu_hist',
            'seed':42
        }
        
        def xgb_f1_score(y_hat, data):
            y_true = data.get_label()
            y_hat = custom_round(y_hat, np.quantile(y_hat, 0.85))
            return 'f1', -f1_score(y_true, y_hat)
    
        oof = np.zeros(len(self.train))
        pred = np.zeros(len(self.test))
        for trn_idx, val_idx in self.kf.split(self.train):
            if self.log:
                train_df = xgb.DMatrix(self.train.loc[trn_idx].drop(['amount_spent','survival_time'], axis=1), label=np.log1p(self.train.loc[trn_idx, self.label]))
                valid_df = xgb.DMatrix(self.train.loc[val_idx].drop(['amount_spent','survival_time'], axis=1), label=np.log1p(self.train.loc[val_idx, self.label]))
            else:
                train_df = xgb.DMatrix(self.train.loc[trn_idx].drop(['amount_spent','survival_time'], axis=1), label=self.train.loc[trn_idx, self.label])
                valid_df = xgb.DMatrix(self.train.loc[val_idx].drop(['amount_spent','survival_time'], axis=1), label=self.train.loc[val_idx, self.label])
            xgb_model = xgb.train(params, train_df, num_boost_round=5000, evals=[(train_df, 'train'), (valid_df, 'val')], verbose_eval=100, early_stopping_rounds=10,feval=xgb_f1_score)
            oof[val_idx] = xgb_model.predict(xgb.DMatrix(self.train.loc[val_idx].drop(['amount_spent','survival_time'], axis=1)))
            pred += xgb_model.predict(xgb.DMatrix(self.test))/self.kf.n_splits
        return oof, pred
    
    def rmse(self, true, pred):
        
        if self.log:
            true = np.expm1(pred)
            mse = mean_squared_error(true, pred)
            rmse = np.round(np.sqrt(mse), 2)
        else:
            mse = mean_squared_error(true, pred)
            rmse = np.round(np.sqrt(mse), 2)
    
        return rmse

In [0]:
FOLDS = 10
SEED = 42

# amount_spent

In [0]:
# test1
as_model1 = model(train=train,test=test1,label='amount_spent' ,log=False)

# randomforest
as_rf_oof1, as_rf_pred1 = as_model1.rf_model()
as_rf_pred_df1 = pd.DataFrame([test1['acc_id'],as_rf_pred1]).transpose()
as_rf_pred_df1.columns = ['acc_id','amount_spent']

# ridge
as_ridge_oof1, as_ridge_pred1 = as_model1.ridge_model()
as_ridge_pred_df1 = pd.DataFrame([test1['acc_id'],as_ridge_oof1]).transpose()
as_ridge_pred_df1.columns = ['acc_id','amount_spent']

# as oof's dataframe1
as_oof_df1 = pd.DataFrame([train['acc_id'],train['amount_spent'],as_rf_oof1,as_ridge_oof1]).transpose()
as_oof_df1.columns = ['acc_id','amount_spent','rf_amount_spent','rid_amount_spent']

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

In [0]:
# test2
as_model2 = model(train=train,test=test2,label = 'amount_spent' ,log=False)

# randomforest
as_rf_oof2, as_rf_pred2 = as_model2.rf_model()
as_rf_pred_df2 = pd.DataFrame([test2['acc_id'],as_rf_pred2]).transpose()
as_rf_pred_df2.columns = ['acc_id','amount_spent']

# ridge
as_ridge_oof2, as_ridge_pred2 = as_model2.ridge_model()
as_ridge_pred_df2 = pd.DataFrame([test2['acc_id'],as_ridge_oof2]).transpose()
as_ridge_pred_df2.columns = ['acc_id','amount_spent']

# as oof's dataframe2
as_oof_df2 = pd.DataFrame([train['acc_id'],train['amount_spent'],as_rf_oof2,as_ridge_oof2]).transpose()
as_oof_df2.columns = ['acc_id','amount_spent','rf_amount_spent','rid_amount_spent']

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

In [0]:
as_rf_pred_df1.loc[as_rf_pred_df1['amount_spent']<0,'amount_spent'] = 0
as_rf_pred_df2.loc[as_rf_pred_df2['amount_spent']<0,'amount_spent'] = 0

as_ridge_pred_df1.loc[as_ridge_pred_df1['amount_spent']<0,'amount_spent'] = 0
as_ridge_pred_df2.loc[as_ridge_pred_df2['amount_spent']<0,'amount_spent'] = 0

# Survival time

In [0]:
# test1
sv_model1 = model(train=train,test=test1,label = 'survival_time' ,log=False)

# randomforest
sv_rf_oof1, sv_rf_pred1 = sv_model1.rf_model()
sv_rf_pred_df1 = pd.DataFrame([test1['acc_id'],sv_rf_pred1.astype(int)]).transpose()
sv_rf_pred_df1.columns = ['acc_id','survival_time']

# ridge
sv_ridge_oof1, sv_ridge_pred1 = sv_model1.ridge_model()
sv_ridge_pred_df1 = pd.DataFrame([test1['acc_id'],sv_ridge_oof1.astype(int)]).transpose()
sv_ridge_pred_df1.columns = ['acc_id','survival_time']

# sv oof's dataframe1
sv_oof_df1 = pd.DataFrame([train['acc_id'],train['survival_time'],sv_rf_oof1.astype(int),sv_ridge_oof1.astype(int)]).transpose()
sv_oof_df1.columns = ['acc_id','survival_time','rf_survival_time','rid_survival_time']


In [0]:
# test2
sv_model2 = model(train=train,test=test2,label = 'survival_time' ,log=False)

# randomforest
sv_rf_oof2, sv_rf_pred2 = sv_model2.rf_model()
sv_rf_pred_df2 = pd.DataFrame([test2['acc_id'],sv_rf_pred2.astype(int)]).transpose()
sv_rf_pred_df2.columns = ['acc_id','survival_time']

# ridge
sv_ridge_oof2, sv_ridge_pred2 = sv_model2.ridge_model()
sv_ridge_pred_df2 = pd.DataFrame([test2['acc_id'],sv_ridge_oof2.astype(int)]).transpose()
sv_ridge_pred_df2.columns = ['acc_id','survival_time']


In [0]:
sv_rf_pred_df1.loc[sv_rf_pred_df1['survival_time']<0,'survival_time'] = 0
sv_rf_pred_df2.loc[sv_rf_pred_df2['survival_time']<0,'survival_time'] = 0

sv_ridge_pred_df1.loc[sv_ridge_pred_df1['survival_time']<0,'survival_time'] = 0
sv_ridge_pred_df2.loc[sv_ridge_pred_df2['survival_time']<0,'survival_time'] = 0

In [0]:
train_label.columns

Index(['acc_id', 'survival_time', 'amount_spent'], dtype='object')

In [0]:
# OOF dataframe
rf_oof = pd.DataFrame({'acc_id':train['acc_id'],'survival_time':sv_rf_oof1.astype(int),'amount_spent':as_rf_oof1})
rf_oof.head()

rid_oof = pd.DataFrame({'acc_id':train['acc_id'],'survival_time':as_ridge_oof1.astype(int),'amount_spent':as_ridge_oof1})
rid_oof.head()

,acc_id,survival_time,amount_spent
0,27835,50,0.056183
1,12351,58,0.198623
2,125437,64,0.152184
3,104483,46,0.068319
4,4704,62,0.197726


,acc_id,survival_time,amount_spent
0,27835,0,0.165782
1,12351,0,0.140505
2,125437,0,0.171106
3,104483,0,0.075382
4,4704,0,0.121095


# Merge

In [0]:
rf_test1 = pd.merge(sv_rf_pred_df1,as_rf_pred_df1,on='acc_id',how='inner')
rf_test2 = pd.merge(sv_rf_pred_df2,as_rf_pred_df2,on='acc_id',how='inner')

ridge_test1 = pd.merge(sv_ridge_pred_df1,as_ridge_pred_df1,on='acc_id',how='inner')
ridge_test2 = pd.merge(sv_ridge_pred_df2,as_ridge_pred_df2,on='acc_id',how='inner')

train_oof = pd.merge(sv_oof_df2,as_oof_df1,on='acc_id')

In [0]:
train_label.describe()

,acc_id,survival_time,amount_spent
count,40000.000000,40000.000000,40000.000000
mean,65281.105550,45.706700,0.124931
std,37525.623536,23.265907,0.722272
min,2.000000,1.000000,0.000000
25%,32792.750000,24.000000,0.000000
50%,65359.000000,64.000000,0.020343
75%,97685.750000,64.000000,0.106119
max,130473.000000,64.000000,39.412632


In [0]:
rf_test1['amount_spent'] = [i/100 for i in rf_test1['amount_spent']]

In [0]:
rf_test2['amount_spent'] = [i/100 for i in rf_test2['amount_spent']]

In [0]:
rf_test1

,acc_id,survival_time,amount_spent
0,7,35,0.023401
1,15,36,0.165975
2,16,27,0.082206
3,18,26,0.149132
4,19,43,0.029035
5,22,36,0.014787
6,24,30,0.051779
7,28,29,0.149756
8,39,34,0.014978
9,51,25,0.123885


In [0]:
rf_test1['acc_id'] = rf_test1['acc_id'].astype(int)
rf_test2['acc_id'] = rf_test2['acc_id'].astype(int)

ridge_test1['acc_id'] = ridge_test1['acc_id'].astype(int)
ridge_test2['acc_id'] = ridge_test2['acc_id'].astype(int)

In [0]:
# output_path = 'drive/My Drive/bigcontest2019/scripts/model/metrics/bjw_inference/'

rf_test1.to_csv('test1_predict.csv', index=False,encoding= 'utf-8')
rf_test2.to_csv('test2_predict.csv', index=False,encoding = 'utf-8')

# ridge_test1.to_csv(output_path+'test1_ridge_predict.csv', index=False)
# ridge_test2.to_csv(output_path+'test2_ridge_predict.csv', index=False)

In [0]:
files.download('test1_predict.csv')
files.download('test2_predict.csv')

In [0]:
train.to_csv('train.csv', index=False)
test1.to_csv('test1.csv', index=False)
test2.to_csv('test2.csv', index=False)

!ls

'data merge'  'explortary data'   model   test1.csv   test2.csv   train.csv


In [0]:
test1.head()

,acc_id,activity_day_nunique,activity_char_id_nunique,activity_server_nunique,activity_playtime_sum,activity_playtime_mean,activity_npc_kill_sum,activity_npc_kill_mean,activity_solo_exp_sum,activity_party_exp_sum,activity_quest_exp_sum,activity_rich_monster_sum,activity_death_sum,activity_revive_sum,activity_exp_recovery_sum,activity_fishing_sum,activity_private_shop_sum,activity_game_money_change_sum,activity_enchant_count_sum,payment_day,payment_amount_spent,combat_day,combat_char_id,combat_server,combat_class,combat_pledge_cnt,combat_random_attacker_cnt,combat_random_defender_cnt,combat_temp_cnt,combat_same_pledge_cnt,combat_etc_cnt,combat_num_opponent,pledge_day,pledge_char_id,pledge_server,pledge_pledge_id,pledge_play_char_cnt,pledge_combat_char_cnt,pledge_pledge_combat_cnt,pledge_random_attacker_cnt,pledge_random_defender_cnt,pledge_same_pledge_cnt,pledge_temp_cnt,pledge_etc_cnt,pledge_combat_play_time,pledge_non_combat_play_time,trade_seller_day_count,trade_seller_type_nunique,trade_seller_type_personal,trade_seller_type_exchange,trade_seller_server_nunique,trade_seller_source_char_id_nunique,trade_seller_target_acc_id_nunique,trade_seller_target_char_id_nunique,trade_seller_item_type_nunique,trade_seller_item_amount_sum,trade_seller_item_price_sum,trade_buyer_day_count,trade_buyer_type_nunique,trade_buyer_type_personal,trade_buyer_type_exchange,trade_buyer_server_nunique,trade_buyer_target_char_id_nunique,trade_buyer_source_acc_id_nunique,trade_buyer_source_char_id_nunique,trade_buyer_item_type_nunique,trade_buyer_item_amount_sum,trade_buyer_item_price_sum
0,7,10,3,2,1.252251,0.104354,0.719474,0.059956,8.624290,0.000000,34.556743,0,0.737650,0.494674,0.0,0.000000,0.000000,0.001291,0.0,1.0,0.035204,10,3,2,2,0.0,0.000000,0.000000,1.918911,0.0,0.000000,0.294386,10.0,2.0,2.0,2.0,19.560411,7.614846,0.506312,19.527071,4.721068,0.000000,12.558147,4.098316,12.579598,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,2.396984e-08,0.002893
1,15,28,3,3,85.705431,1.032596,18.207875,0.219372,0.630317,0.000000,0.000000,0,7.868271,6.183422,0.0,0.000000,0.000000,0.023029,0.0,0.0,0.000000,28,3,3,2,0.0,0.000000,39.041128,0.000000,0.0,2.027007,2.256961,28.0,3.0,3.0,3.0,99.317807,15.410139,0.000000,0.000000,48.355185,0.000000,4.054205,1.797879,107.054983,16.478587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000
2,16,3,7,1,0.081923,0.009103,0.000000,0.000000,0.000000,0.000000,0.100846,0,0.000000,0.000000,0.0,0.000000,0.000000,0.034526,0.0,1.0,0.035204,3,7,1,2,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,3.0,7.0,1.0,8.0,1.190948,0.036089,0.000000,0.000000,0.000000,0.000000,0.000000,0.007078,0.074364,0.056062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000
3,18,13,1,1,40.008820,3.077602,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,36.945877,0.921976,0.0,0.0,0.000000,13,1,1,1,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,48.0,2.0,47.0,1.0,1.0,1.0,32.0,32.0,3.0,0.304424,2.569245,5.0,1.0,0.0,5.0,1.0,1.0,1.0,1.0,3.0,4.817938e-06,0.000000
4,19,28,4,4,84.139533,2.052184,79.866399,1.947961,0.271431,0.954236,0.007157,2,0.737650,0.494674,0.0,37.465503,0.000000,1.863396,0.0,0.0,0.000000,28,4,4,3,0.0,3.131975,9.464516,1.918911,0.0,2.815288,4.219536,28.0,2.0,4.0,1.0,17.034159,2.526252,0.000000,0.710075,1.430627,1.591434,5.339685,0.481322,10.912210,2.733728,12.0,1.0,0.0,12.0,1.0,1.0,10.0,10.0,4.0,4.077989,0.000000,53.0,2.0,36.0,17.0,1.0,1.0,29.0,29.0,5.0,4.237634e+00,2.005447


In [0]:
from google.colab import files

files.download('train.csv')
files.download('test1.csv')
files.download('test2.csv')

# Score

In [0]:
score_path = 'drive/My Drive/bigcontest2019/scripts'
os.chdir(score_path)
from model.metrics import score_function

In [0]:
score_function.score_function(train_label, train_label, path=False) # 91693.76240883442
print()
score_function.score_function(rf_oof, train_label, path=False)

91693.76240883442


91693.76240883442


10780.063262817888


10780.063262817888

In [0]:
# from google.colab import files

# output_path = 'drive/My Drive/bigcontest2019/scripts/model/metrics/bjw_inference/'
# os.chdir(output_path)

# files.download('test1_rf_predict.csv')
# files.download('test2_rf_predict.csv')
